In [142]:
import pandas as pd
import numpy as np
import json
import requests
import plotly.express as px
import matplotlib.pyplot as plt
import datetime
import time
from bs4 import BeautifulSoup
from bs4 import BeautifulStoneSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import tempfile
import time
from bs4 import BeautifulSoup

In [143]:
mobile=pd.read_csv(r"D:\JUPYTER NOTEBOOK--1\PROJECT-1\DATA-ANALYSIS\smartphones - smartphones.csv")
mobile.head()

,model,price,rating,sim,processor,ram,battery,display,camera,card,os
0,OnePlus 11 5G,"₹54,999",89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,OnePlus Nord CE 2 Lite 5G,"₹19,989",81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
2,Samsung Galaxy A14 5G,"₹16,499",75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Exynos 1330, Octa Core, 2.4 GHz Processor","4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,"Memory Card Supported, upto 1 TB",Android v13
3,Motorola Moto G62 5G,"₹14,999",81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
4,Realme 10 Pro Plus,"₹24,999",82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Dimensity 1080, Octa Core, 2.6 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Not Supported,Android v13


## Data Assessing

### Quality Issues

1. **model** - some brands are written diiferently like OPPO in model column `consistency`
2. **price** - has unneccesary '₹' `validity`
3. **price** - has ',' between numbers `validity`
4. **price** - phone Namotel has a price of 99 `accuracy`
5. **ratings** - missing values `completeness`
6. **processor** - has some incorrect values for some samsung phones(row # -642,647,649,659,667,701,750,759,819,859,883,884,919,927,929,932,1002) `validity`
7. There is ipod on row 756 `validity`
8. **memory** - incorrect values in rows (441,485,534,553,584,610,613,642,647,649,659,667,701,750,759,819,859,884,919,927,929,932,990,1002) `validity`
9. **battery** - incorrect values in rows(113,151,309,365,378,441,450,553,584,610,613,630,642,647,649,659,667,701,750,756,759,764,819,855,859,884,915,916,927,929,932,990,1002) `validity`
10. **display** - sometimes frequency is not available `completeness`
11. **display** - incorrect values in rows(378,441,450,553,584,610,613,630,642,647,649,659,667,701,750,759,764,819,859,884,915,916,927,929,932,990,1002) `validity`
12. certain phones are foldable and the info is scattered `validity`
13. **camera** - words like Dual, Triple and Quad are used to represent number of cameras and front and rear cameras are separated by '&'
14. **camera** - problem with rows (100,113,151,157,161,238,273,308,309,323,324,365,367,378,394,441,450,484,506,534,553,571,572,575,584,610,613,615,630,642,647,649,659,667,684,687,705,711,723,728,750,756,759,764,792,819,846,854,855,858,883,884,896,915,916,927,929,932,945,956,990,995,1002,1016
) `validity`
15. **card** - sometimes contains info about os and camera `validity`
16. **os** - sometimes contains info about bluetooth and fm radio `validity`
17. **os** - issue with rows (324,378) `validity`
18. **os** - sometimes contains os version name like lollipop `consistency`
19. missing values in camera, card and os `completeness`
20. datatype  of price and rating is incorrect `validity`



### Tidiness Issues

1. **sim** - can be split into 3 cols has_5g, has_NFC, has_IR_Blaster
2. **ram** - can be split into 2 cols RAM and ROM
3. **processor** - can be split into processor name, cores and cpu speed.
4. **battery** - can be split into battery capacity, fast_charging_available
5. **display** - can be split into size, resolution_width, resolution_height and frequency
6. **camera** - can be split into front and rear camera
7. **card** - can be split into supported, extended_upto

In [6]:
df=mobile.copy()


In [7]:
df['price']=df['price'].str.replace('₹','').str.replace(',','').astype('int')

In [8]:
df=df.reset_index()
df['index']=df['index']+2


In [9]:
processor_rows = set((642,647,649,659,667,701,750,759,819,859,883,884,919,927,929,932,1002))
ram_rows = set((441,485,534,553,584,610,613,642,647,649,659,667,701,750,759,819,859,884,919,927,929,932,990,1002))
battery_rows = set((113,151,309,365,378,441,450,553,584,610,613,630,642,647,649,659,667,701,750,756,759,764,819,855,859,884,915,916,927,929,932,990,1002))
display_rows = set((378,441,450,553,584,610,613,630,642,647,649,659,667,701,750,759,764,819,859,884,915,916,927,929,932,990,1002))
camera_rows = set((100,113,151,157,161,238,273,308,309,323,324,365,367,378,394,441,450,484,506,534,553,571,572,575,584,610,613,615,630,642,647,649,659,667,684,687,705,711,723,728,750,756,759,764,792,819,846,854,855,858,883,884,896,915,916,927,929,932,945,956,990,995,1002,1016 ))

In [10]:
df[df['index'].isin(processor_rows | ram_rows | battery_rows | display_rows | camera_rows)]

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
98,100,Vivo X Fold 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","12 GB RAM, 256 GB inbuilt",4600 mAh Battery with 66W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display",Foldable Display,50 MP Quad Rear & 16 MP Front Camera,Android v12
111,113,Apple iPhone 12,51999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
149,151,Apple iPhone 12 Mini,40999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
155,157,Nokia 2780 Flip,4990,NaN,"Dual Sim, 3G, 4G, Wi-Fi","Snapdragon QM215, Quad Core, 1.3 GHz Processor","4 GB RAM, 512 MB inbuilt",1450 mAh Battery,"2.7 inches, 240 x 320 px Display",Dual Display,5 MP Rear Camera,"Memory Card Supported, upto 32 GB"
159,161,Oppo Find N2 5G,94990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4520 mAh Battery with 67W Fast Charging,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...","Foldable Display, Dual Display",50 MP + 48 MP + 32 MP Triple Rear & 32 MP + 32...,Memory Card Not Supported
...,...,...,...,...,...,...,...,...,...,...,...,...
954,956,Vivo X Fold 5G (12GB RAM + 512GB),118990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","12 GB RAM, 512 GB inbuilt",4600 mAh Battery with 66W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display",Foldable Display,50 MP Quad Rear & 16 MP Front Camera,Android v12
988,990,Nokia 5310 Dual Sim,3399,NaN,Dual Sim,"8 MB RAM, 16 MB inbuilt",1200 mAh Battery,"2.4 inches, 240 x 320 px Display",0.3 MP Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,Browser
993,995,Huawei Mate X,169000,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...","Kirin 990, Octa Core, 2.86 GHz Processor","8 GB RAM, 512 GB inbuilt",4500 mAh Battery with 55W Fast Charging,"8 inches, 2200 x 2480 px Display",Foldable Display,48 MP Quad Rear Camera,"Memory Card (Hybrid), upto 256 GB"
1000,1002,XTouch F40 Flip,1999,NaN,Dual Sim,No 3G,No Wifi,"32 MB RAM, 32 MB inbuilt",800 mAh Battery,"1.77 inches, 240 x 320 px Display",Dual Display,1.3 MP Rear Camera


In [11]:
df[df['index'].isin(processor_rows & ram_rows & battery_rows & display_rows & camera_rows)]

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
640,642,Nokia 105 Plus,1299,NaN,Dual Sim,"4 MB RAM, 4 MB inbuilt",800 mAh Battery,"1.77 inches, 128 x 160 px Display",No Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,NaN
645,647,Nokia 2760 Flip,5490,NaN,"Dual Sim, 3G, 4G, Wi-Fi",1450 mAh Battery,"3.6 inches, 240 x 320 px Display",5 MP Rear & 5 MP Front Camera,"Memory Card Supported, upto 32 GB",Kaios v3.0,Bluetooth,NaN
647,649,Motorola Moto A10,1339,NaN,Dual Sim,"4 MB RAM, 4 MB inbuilt",1750 mAh Battery,"1.8 inches, 160 x 128 px Display",No Rear Camera,"Memory Card Supported, upto 32 GB",NaN,NaN
657,659,Zanco Tiny T1,2799,NaN,Single Sim,"32 MB RAM, 32 MB inbuilt",200 mAh Battery,"0.49 inches, 64 x 32 px Display",No Rear Camera,No FM Radio,Bluetooth,NaN
665,667,itel it2163S,958,NaN,Dual Sim,"4 MB RAM, 4 MB inbuilt",1200 mAh Battery,"1.8 inches, 160 x 128 px Display",No Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,NaN
748,750,Nokia 400 4G,3290,NaN,"Dual Sim, 4G, VoLTE, Wi-Fi",2000 mAh Battery,"2.4 inches, 240 x 320 px Display",0.3 MP Rear & 0.3 MP Front Camera,"Memory Card Supported, upto 64 GB",Bluetooth,Browser,NaN
757,759,Karbonn KU3i,995,NaN,Dual Sim,"52 MB RAM, 32 MB inbuilt",1000 mAh Battery,"1.8 inches, 128 x 160 px Display",No Rear Camera,"Memory Card Supported, upto 16 GB",Bluetooth,NaN
817,819,itel Magic X,2239,NaN,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi",No 3G,T117,"48 MB RAM, 128 MB inbuilt",1200 mAh Battery,"2.4 inches, 240 x 320 px Display",1.3 MP Rear Camera,"Memory Card Supported, upto 64 GB"
882,884,Nokia 5710 XpressAudio,4799,NaN,"Dual Sim, 3G, 4G",No Wifi,Unisoc T107,"48 MB RAM, 128 MB inbuilt",1450 mAh Battery,"2.4 inches, 240 x 320 px Display",0.3 MP Rear Camera,"Memory Card Supported, upto 32 GB"
925,927,Nokia 3310 4G,3999,NaN,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi","256 MB RAM, 512 MB inbuilt",1200 mAh Battery,"2.4 inches, 240 x 320 px Display",2 MP Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,Browser


In [12]:
df=df[df['price']>3400]

In [13]:
df[df['index'].isin(processor_rows)]

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
645,647,Nokia 2760 Flip,5490,NaN,"Dual Sim, 3G, 4G, Wi-Fi",1450 mAh Battery,"3.6 inches, 240 x 320 px Display",5 MP Rear & 5 MP Front Camera,"Memory Card Supported, upto 32 GB",Kaios v3.0,Bluetooth,NaN
857,859,LG Folder 2,11999,NaN,"Single Sim, 3G, 4G, Wi-Fi","1 GB RAM, 8 GB inbuilt",1470 mAh Battery,"2.8 inches, 240 x 320 px Display",2 MP Rear Camera,Memory Card Supported,Bluetooth,NaN
882,884,Nokia 5710 XpressAudio,4799,NaN,"Dual Sim, 3G, 4G",No Wifi,Unisoc T107,"48 MB RAM, 128 MB inbuilt",1450 mAh Battery,"2.4 inches, 240 x 320 px Display",0.3 MP Rear Camera,"Memory Card Supported, upto 32 GB"
925,927,Nokia 3310 4G,3999,NaN,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi","256 MB RAM, 512 MB inbuilt",1200 mAh Battery,"2.4 inches, 240 x 320 px Display",2 MP Rear Camera,"Memory Card Supported, upto 32 GB",Bluetooth,Browser


In [14]:
df.drop([645,857,882,925],inplace=True)

In [15]:
df[df['index'].isin(ram_rows)]

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
439,441,Apple iPhone SE 3 2022,43900,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A15, Hexa Core, 3.22 GHz Processor",64 GB inbuilt,"4.7 inches, 750 x 1334 px Display",12 MP Rear & 7 MP Front Camera,Memory Card Not Supported,iOS v15,No FM Radio
483,485,Huawei Mate 50 RS Porsche Design,239999,81.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi, NFC, IR Blaster","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor",512 GB inbuilt,4700 mAh Battery with 66W Fast Charging,"6.74 inches, 1212 x 2616 px, 120 Hz Display",50 MP + 48 MP + 13 MP Triple Rear & 13 MP Fron...,"Memory Card (Hybrid), upto 256 GB",Hongmeng OS v3.0
582,584,Nokia 8210 4G,3749,NaN,"Dual Sim, 3G, 4G",No Wifi,Unisoc T107,"48 MB RAM, 128 MB inbuilt",1450 mAh Battery,"2.8 inches, 240 x 320 px Display",0.3 MP Rear Camera,"Memory Card Supported, upto 32 GB"


In [16]:
df.drop(582,inplace=True)

In [17]:
df[df['index'].isin(battery_rows)]

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
111,113,Apple iPhone 12,51999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
149,151,Apple iPhone 12 Mini,40999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
307,309,Apple iPhone 12 (128GB),55999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 128 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
363,365,Apple iPhone 12 Mini (128GB),45999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 128 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio
376,378,Nokia 2660 Flip,4649,NaN,"Dual Sim, 3G, 4G",No Wifi,Unisoc T107,"48 MB RAM, 128 MB inbuilt",1450 mAh Battery,"2.8 inches, 240 x 320 px Display",Dual Display,0.3 MP Rear Camera
439,441,Apple iPhone SE 3 2022,43900,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A15, Hexa Core, 3.22 GHz Processor",64 GB inbuilt,"4.7 inches, 750 x 1334 px Display",12 MP Rear & 7 MP Front Camera,Memory Card Not Supported,iOS v15,No FM Radio
448,450,Apple iPhone 15 Pro,130990,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",Bionic A16,"8 GB RAM, 128 GB inbuilt","6.06 inches, 1170 x 2532 px, 120 Hz Display wi...",50 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v15,No FM Radio
628,630,Apple iPhone 12 Pro (512GB),139900,80.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","6 GB RAM, 512 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v14.0,No FM Radio
754,756,Apple iPod Touch (7th Gen),18900,NaN,Wi-Fi,32 GB inbuilt,"4 inches, 640 x 1136 px Display",8 MP Rear & 1.2 MP Front Camera,iOS v12,No FM Radio,Bluetooth,Browser
762,764,Apple iPhone SE 4,49990,60.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A15, Hexa Core, 3.22 GHz Processor",64 GB inbuilt,"6.1 inches, 750 x 1580 px Display",12 MP Rear & 10.8 MP Front Camera,Memory Card Not Supported,iOS v16,No FM Radio


In [18]:
df.drop([376,754],inplace=True)

In [19]:
temp_df=df[df['index'].isin(battery_rows)]

In [20]:
x=temp_df.iloc[:,7:].shift(1,axis=1).values

In [21]:
df.loc[temp_df.index,temp_df.columns[7:]]=x

In [22]:
df.drop([155,271],inplace=True)

In [23]:
df[df['index'].isin(camera_rows)]

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
98,100,Vivo X Fold 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","12 GB RAM, 256 GB inbuilt",4600 mAh Battery with 66W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display",Foldable Display,50 MP Quad Rear & 16 MP Front Camera,Android v12
111,113,Apple iPhone 12,51999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt",None,"6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14
149,151,Apple iPhone 12 Mini,40999,74.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 64 GB inbuilt",None,"5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14
159,161,Oppo Find N2 5G,94990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4520 mAh Battery with 67W Fast Charging,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...","Foldable Display, Dual Display",50 MP + 48 MP + 32 MP Triple Rear & 32 MP + 32...,Memory Card Not Supported
236,238,Xiaomi Mix Fold 2 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...","Snapdragon 8+ Gen1 , Octa Core, 3.2 GHz Proce...","12 GB RAM, 256 GB inbuilt",4500 mAh Battery with 67W Fast Charging,"8.02 inches, 1914 x 2160 px, 120 Hz Display wi...","Foldable Display, Dual Display",50 MP + 13 MP + 8 MP Triple Rear & 20 MP Front...,Android v12
306,308,Samsung Galaxy Z Flip 3,69999,84.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 888, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",3300 mAh Battery with 15W Fast Charging,"6.7 inches, 1080 x 2640 px, 120 Hz Display wit...","Foldable Display, Dual Display",12 MP + 12 MP Dual Rear & 10 MP Front Camera,Memory Card Not Supported
307,309,Apple iPhone 12 (128GB),55999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 128 GB inbuilt",None,"6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14
321,323,Samsung Galaxy Z Fold 4,154998,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4400 mAh Battery with 25W Fast Charging,"7.6 inches, 1812 x 2176 px, 120 Hz Display wit...","Foldable Display, Dual Display",50 MP + 12 MP + 10 MP Triple Rear & 10 MP + 4 ...,Android v12
322,324,Royole FlexPai 2,109999,87.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 865, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",4450 mAh Battery,"7.8 inches, 1440 x 1920 px Display","Foldable Display, Dual Display",64 MP + 16 MP + 8 MP Triple Rear & 32 MP Front...,"Memory Card Supported, upto 256 GB"
363,365,Apple iPhone 12 Mini (128GB),45999,75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Bionic A14, Hexa Core, 3.1 GHz Processor","4 GB RAM, 128 GB inbuilt",None,"5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14


In [24]:
temp_df=df[~df['camera'].str.contains('MP')]
temp_df

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
69,71,Oppo Find N Fold,99990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","8 GB RAM, 256 GB inbuilt",5000 mAh Battery with 67W Fast Charging,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...","Foldable Display, Dual Display",64 MP + 10 MP + 8 MP Triple Rear & 32 MP Front...,Android v12
98,100,Vivo X Fold 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","12 GB RAM, 256 GB inbuilt",4600 mAh Battery with 66W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display",Foldable Display,50 MP Quad Rear & 16 MP Front Camera,Android v12
159,161,Oppo Find N2 5G,94990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4520 mAh Battery with 67W Fast Charging,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...","Foldable Display, Dual Display",50 MP + 48 MP + 32 MP Triple Rear & 32 MP + 32...,Memory Card Not Supported
236,238,Xiaomi Mix Fold 2 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...","Snapdragon 8+ Gen1 , Octa Core, 3.2 GHz Proce...","12 GB RAM, 256 GB inbuilt",4500 mAh Battery with 67W Fast Charging,"8.02 inches, 1914 x 2160 px, 120 Hz Display wi...","Foldable Display, Dual Display",50 MP + 13 MP + 8 MP Triple Rear & 20 MP Front...,Android v12
306,308,Samsung Galaxy Z Flip 3,69999,84.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 888, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",3300 mAh Battery with 15W Fast Charging,"6.7 inches, 1080 x 2640 px, 120 Hz Display wit...","Foldable Display, Dual Display",12 MP + 12 MP Dual Rear & 10 MP Front Camera,Memory Card Not Supported
321,323,Samsung Galaxy Z Fold 4,154998,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4400 mAh Battery with 25W Fast Charging,"7.6 inches, 1812 x 2176 px, 120 Hz Display wit...","Foldable Display, Dual Display",50 MP + 12 MP + 10 MP Triple Rear & 10 MP + 4 ...,Android v12
322,324,Royole FlexPai 2,109999,87.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 865, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",4450 mAh Battery,"7.8 inches, 1440 x 1920 px Display","Foldable Display, Dual Display",64 MP + 16 MP + 8 MP Triple Rear & 32 MP Front...,"Memory Card Supported, upto 256 GB"
365,367,OPPO Find N Flip,89990,88.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Dimensity 9000, Octa Core, 3.05 GHz Processor","8 GB RAM, 128 GB inbuilt",4300 mAh Battery with 44W Fast Charging,"6.8 inches, 1200 x 2400 px, 120 Hz Display wit...","Foldable Display, Dual Display",50 MP + 8 MP Dual Rear & 32 MP Front Camera,Memory Card Not Supported
392,394,OPPO Find N2 Flip,70990,88.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Dimensity 9000 Plus, Octa Core, 3.2 GHz Processor","8 GB RAM, 256 GB inbuilt",4300 mAh Battery with 44W Fast Charging,"6.8 inches, 1080 x 2520 px, 120 Hz Display wit...","Foldable Display, Dual Display",50 MP + 8 MP Dual Rear & 32 MP Front Camera,Memory Card Not Supported
482,484,Vivo X Fold Plus,113990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4730 mAh Battery with 80W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display wi...","Foldable Display, Dual Display",50 MP Quad Rear & 16 MP + 16 MP Dual Front Camera,Android v12


In [25]:
df.loc[temp_df.index,'camera']=temp_df['card'].values

In [26]:
df.loc[temp_df.index]

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
69,71,Oppo Find N Fold,99990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","8 GB RAM, 256 GB inbuilt",5000 mAh Battery with 67W Fast Charging,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...",64 MP + 10 MP + 8 MP Triple Rear & 32 MP Front...,64 MP + 10 MP + 8 MP Triple Rear & 32 MP Front...,Android v12
98,100,Vivo X Fold 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen1, Octa Core, 3 GHz Processor","12 GB RAM, 256 GB inbuilt",4600 mAh Battery with 66W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display",50 MP Quad Rear & 16 MP Front Camera,50 MP Quad Rear & 16 MP Front Camera,Android v12
159,161,Oppo Find N2 5G,94990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4520 mAh Battery with 67W Fast Charging,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 32 MP + 32...,50 MP + 48 MP + 32 MP Triple Rear & 32 MP + 32...,Memory Card Not Supported
236,238,Xiaomi Mix Fold 2 5G,106990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...","Snapdragon 8+ Gen1 , Octa Core, 3.2 GHz Proce...","12 GB RAM, 256 GB inbuilt",4500 mAh Battery with 67W Fast Charging,"8.02 inches, 1914 x 2160 px, 120 Hz Display wi...",50 MP + 13 MP + 8 MP Triple Rear & 20 MP Front...,50 MP + 13 MP + 8 MP Triple Rear & 20 MP Front...,Android v12
306,308,Samsung Galaxy Z Flip 3,69999,84.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 888, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",3300 mAh Battery with 15W Fast Charging,"6.7 inches, 1080 x 2640 px, 120 Hz Display wit...",12 MP + 12 MP Dual Rear & 10 MP Front Camera,12 MP + 12 MP Dual Rear & 10 MP Front Camera,Memory Card Not Supported
321,323,Samsung Galaxy Z Fold 4,154998,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4400 mAh Battery with 25W Fast Charging,"7.6 inches, 1812 x 2176 px, 120 Hz Display wit...",50 MP + 12 MP + 10 MP Triple Rear & 10 MP + 4 ...,50 MP + 12 MP + 10 MP Triple Rear & 10 MP + 4 ...,Android v12
322,324,Royole FlexPai 2,109999,87.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 865, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",4450 mAh Battery,"7.8 inches, 1440 x 1920 px Display",64 MP + 16 MP + 8 MP Triple Rear & 32 MP Front...,64 MP + 16 MP + 8 MP Triple Rear & 32 MP Front...,"Memory Card Supported, upto 256 GB"
365,367,OPPO Find N Flip,89990,88.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Dimensity 9000, Octa Core, 3.05 GHz Processor","8 GB RAM, 128 GB inbuilt",4300 mAh Battery with 44W Fast Charging,"6.8 inches, 1200 x 2400 px, 120 Hz Display wit...",50 MP + 8 MP Dual Rear & 32 MP Front Camera,50 MP + 8 MP Dual Rear & 32 MP Front Camera,Memory Card Not Supported
392,394,OPPO Find N2 Flip,70990,88.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Dimensity 9000 Plus, Octa Core, 3.2 GHz Processor","8 GB RAM, 256 GB inbuilt",4300 mAh Battery with 44W Fast Charging,"6.8 inches, 1080 x 2520 px, 120 Hz Display wit...",50 MP + 8 MP Dual Rear & 32 MP Front Camera,50 MP + 8 MP Dual Rear & 32 MP Front Camera,Memory Card Not Supported
482,484,Vivo X Fold Plus,113990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4730 mAh Battery with 80W Fast Charging,"8.03 inches, 1916 x 2160 px, 120 Hz Display wi...",50 MP Quad Rear & 16 MP + 16 MP Dual Front Camera,50 MP Quad Rear & 16 MP + 16 MP Dual Front Camera,Android v12


In [27]:
temp_df=df[df['card'].str.contains('MP')]

In [28]:
df['card'].value_counts()
# Memory Card Not Supported 

card
Memory Card Supported, upto 1 TB                                       171
Memory Card Not Supported                                              123
Android v12                                                            107
Memory Card Supported, upto 512 GB                                     105
Memory Card (Hybrid), upto 1 TB                                         91
Memory Card Supported                                                   89
Memory Card Supported, upto 256 GB                                      87
Android v13                                                             46
Android v11                                                             41
Memory Card (Hybrid)                                                    30
Memory Card (Hybrid), upto 256 GB                                       13
Android v10                                                             11
Memory Card (Hybrid), upto 512 GB                                       11
Memory Card Supporte

In [29]:
df.loc[temp_df.index,'card']='Memory Card Not Supported '
# df.loc[temp_df.index,'card']

In [30]:
temp_df=df[(df['card'].str.contains('v')) | (df['card'].str.contains('OS'))]


In [31]:
temp_df=df[~df['card'].str.contains('Memory Card')]


In [32]:
df.loc[temp_df.index,'os']=temp_df['card'].values

In [33]:
df.loc[temp_df.index,'card']= 'Memory Card Not Supported' 

In [34]:
df['os'].value_counts()

os
Android v12                           394
Android v11                           274
Android v13                            91
Android v10                            69
Android v9.0 (Pie)                     29
Android v10.0                          23
iOS v16                                15
iOS v15                                12
Android v8.1 (Oreo)                    10
iOS v14                                 6
Memory Card Not Supported               6
Android v11.0                           4
Android v8.0 (Oreo)                     4
iOS v13                                 4
iOS v15.0                               3
Android v6.0 (Marshmallow)              2
Memory Card (Hybrid), upto 256 GB       2
Memory Card (Hybrid), upto 2 TB         2
HarmonyOS v2.0                          2
Android v5.1.1 (Lollipop)               2
Harmony v2.0                            2
EMUI v12                                2
Memory Card Supported, upto 256 GB      2
Android v12.1                  

In [35]:
df[
    ~((df['os'].str.contains('Android', na=False)) |
    (df['os'].str.contains('iOS', na=False)) |
    (df['os'].str.contains('v', na=False)) |
    (df['os'].str.contains('OS', na=False)))
].shape


(15, 12)

In [36]:
df[~(df['os'].str.contains('v') | df['os'].str.contains('OS') | df['os'].str.contains('Android'))]

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
159,161,Oppo Find N2 5G,94990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",4520 mAh Battery with 67W Fast Charging,"7.1 inches, 1792 x 1920 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 32 MP + 32...,Memory Card Not Supported,Memory Card Not Supported
306,308,Samsung Galaxy Z Flip 3,69999,84.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 888, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",3300 mAh Battery with 15W Fast Charging,"6.7 inches, 1080 x 2640 px, 120 Hz Display wit...",12 MP + 12 MP Dual Rear & 10 MP Front Camera,Memory Card Not Supported,Memory Card Not Supported
322,324,Royole FlexPai 2,109999,87.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 865, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",4450 mAh Battery,"7.8 inches, 1440 x 1920 px Display",64 MP + 16 MP + 8 MP Triple Rear & 32 MP Front...,Memory Card Not Supported,"Memory Card Supported, upto 256 GB"
365,367,OPPO Find N Flip,89990,88.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Dimensity 9000, Octa Core, 3.05 GHz Processor","8 GB RAM, 128 GB inbuilt",4300 mAh Battery with 44W Fast Charging,"6.8 inches, 1200 x 2400 px, 120 Hz Display wit...",50 MP + 8 MP Dual Rear & 32 MP Front Camera,Memory Card Not Supported,Memory Card Not Supported
392,394,OPPO Find N2 Flip,70990,88.0,"Single Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Dimensity 9000 Plus, Octa Core, 3.2 GHz Processor","8 GB RAM, 256 GB inbuilt",4300 mAh Battery with 44W Fast Charging,"6.8 inches, 1080 x 2520 px, 120 Hz Display wit...",50 MP + 8 MP Dual Rear & 32 MP Front Camera,Memory Card Not Supported,Memory Card Not Supported
486,488,Nokia 8000 4G,6899,NaN,"Dual Sim, 3G, 4G, Wi-Fi","Snapdragon 210, Quad Core, 1.1 GHz Processor","512 MB RAM, 4 GB inbuilt",1500 mAh Battery,"2.8 inches, 240 x 320 px Display",2 MP Rear Camera,Memory Card Supported,Bluetooth
504,506,Samsung Galaxy Z Flip 4 5G,89999,88.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor","8 GB RAM, 128 GB inbuilt",3700 mAh Battery with 25W Fast Charging,"6.7 inches, 1080 x 2640 px, 120 Hz Display wit...",12 MP + 12 MP Dual Rear & 10 MP Front Camera,Memory Card Not Supported,Memory Card Not Supported
570,572,LG Wing 5G,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 765G , Octa Core, 2.4 GHz Processor","8 GB RAM, 128 GB inbuilt",4000 mAh Battery with Fast Charging,"6.8 inches, 1080 x 2460 px Display",64 MP + 13 MP + 12 MP Triple Rear & 32 MP Fron...,Memory Card Not Supported,"Memory Card (Hybrid), upto 2 TB"
613,615,Oukitel WP21,22990,82.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi","Helio G99, Octa Core, 2.2 GHz Processor","12 GB RAM, 256 GB inbuilt",9800 mAh Battery with 66W Fast Charging,"6.78 inches, 1080 x 2400 px, 120 Hz Display",64 MP + 20 MP + 2 MP Triple Rear & Main Front ...,Memory Card Not Supported,Memory Card (Hybrid)
682,684,LG V60 ThinQ,79990,NaN,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 865, Octa Core, 2.84 GHz Processor","8 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,"6.8 inches, 1080 x 2460 px Display with Punch ...",64 MP + 13 MP + 0.3 MP Triple Rear & 10 MP Fro...,Memory Card Not Supported,"Memory Card (Hybrid), upto 2 TB"


In [37]:
temp_df=df.loc[[322,570,613,682,726,844,894,993],'os']

In [38]:
df.loc[temp_df.index,'card']=temp_df.values

In [39]:
df[df['os']=='Bluetooth']

,index,model,price,rating,sim,processor,ram,battery,display,camera,card,os
486,488,Nokia 8000 4G,6899,NaN,"Dual Sim, 3G, 4G, Wi-Fi","Snapdragon 210, Quad Core, 1.1 GHz Processor","512 MB RAM, 4 GB inbuilt",1500 mAh Battery,"2.8 inches, 240 x 320 px Display",2 MP Rear Camera,Memory Card Supported,Bluetooth


In [40]:
df.loc[df['os'] == 'Bluetooth', 'os'] = np.nan


In [41]:
df['os'].value_counts()

os
Android v12                           394
Android v11                           274
Android v13                            91
Android v10                            69
Android v9.0 (Pie)                     29
Android v10.0                          23
iOS v16                                15
iOS v15                                12
Android v8.1 (Oreo)                    10
iOS v14                                 6
Memory Card Not Supported               6
Android v11.0                           4
Android v8.0 (Oreo)                     4
iOS v13                                 4
iOS v15.0                               3
Android v6.0 (Marshmallow)              2
Memory Card (Hybrid), upto 256 GB       2
HarmonyOS v2.0                          2
Memory Card (Hybrid), upto 2 TB         2
Android v5.1.1 (Lollipop)               2
Harmony v2.0                            2
EMUI v12                                2
Memory Card Supported, upto 256 GB      2
Android v12.1                  

In [42]:
temp_df=df[~(df['os'].str.contains('v') | df['os'].str.contains('OS') | df['os'].str.contains('Android'))]
# temp_df

In [43]:
df.loc[temp_df.index,'os']=np.nan


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 982 entries, 0 to 1019
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      982 non-null    int64  
 1   model      982 non-null    object 
 2   price      982 non-null    int32  
 3   rating     879 non-null    float64
 4   sim        982 non-null    object 
 5   processor  982 non-null    object 
 6   ram        982 non-null    object 
 7   battery    971 non-null    object 
 8   display    982 non-null    object 
 9   camera     982 non-null    object 
 10  card       982 non-null    object 
 11  os         967 non-null    object 
dtypes: float64(1), int32(1), int64(1), object(9)
memory usage: 128.2+ KB


In [45]:
df.drop(columns='index',inplace=True)


In [46]:
df.to_csv('final_phone_dataset_1.csv',index=False)

In [47]:
final_df=pd.read_csv('final_phone_dataset_1.csv')
# final_df.head()


In [48]:
brand_names=final_df['model'].str.split().str.get(0).str.lower()
final_df.insert(1,'brand_name',brand_names)


In [49]:
final_df['brand_name'].value_counts().head()
final_df.iloc[979]['sim']

'Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Blaster'

In [50]:
final_df['sim'].str.split().str.get(4)

0      5G,
1      5G,
2      5G,
3      5G,
4      5G,
      ... 
977    5G,
978    5G,
979    5G,
980    5G,
981    5G,
Name: sim, Length: 982, dtype: object

In [51]:
has_5g=final_df['sim'].str.contains('5G')
has_nfc=final_df['sim'].str.contains('NFC')
has_blaster=final_df['sim'].str.contains('IR Blaster')

In [52]:
final_df.insert(6,'has_5g',has_5g)
final_df.insert(7,'has_nfc',has_nfc)
final_df.insert(8,'has_blaster',has_blaster)


In [53]:
processor_name=final_df['processor'].str.split(',').str.get(0)
num_core=final_df['processor'].str.split(',').str.get(1)
processor_speed=final_df['processor'].str.split(',').str.get(2)

In [54]:
final_df.insert(10,'processor_name',processor_name)
final_df.insert(11,'num_of_cores',num_core)
final_df.insert(12,'processor_speed',processor_speed)

In [55]:
final_df.head(2)

,model,brand_name,price,rating,sim,processor,has_5g,has_nfc,has_blaster,ram,processor_name,num_of_cores,processor_speed,battery,display,camera,card,os
0,OnePlus 11 5G,oneplus,54999,89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor",True,True,False,"12 GB RAM, 256 GB inbuilt",Snapdragon 8 Gen2,Octa Core,3.2 GHz Processor,5000 mAh Battery with 100W Fast Charging,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,OnePlus Nord CE 2 Lite 5G,oneplus,19989,81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor",True,False,False,"6 GB RAM, 128 GB inbuilt",Snapdragon 695,Octa Core,2.2 GHz Processor,5000 mAh Battery with 33W Fast Charging,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12


In [56]:
temp_df = final_df[final_df['processor_name'].str.contains('Core')][['processor_name', 'num_of_cores',	'processor_speed']].shift(1,axis=1)

In [57]:
temp_df

,processor_name,num_of_cores,processor_speed
118,None,Octa Core Processor,NaN
143,None,Quad Core,1.4 GHz Processor
187,None,Quad Core,1.6 GHz Processor
200,None,Octa Core,2 GHz Processor
307,None,Octa Core,2 GHz Processor
313,None,Quad Core,1.3 GHz Processor
491,None,Octa Core Processor,NaN
524,None,Octa Core Processor,NaN
576,None,Octa Core,2 GHz Processor
735,None,Quad Core,1.6 GHz Processor


In [58]:
final_df.loc[temp_df.index,['processor_name', 'num_of_cores',	'processor_speed']] = temp_df.values

In [59]:
final_df[final_df['processor_name']=='(28 nm)']

,model,brand_name,price,rating,sim,processor,has_5g,has_nfc,has_blaster,ram,processor_name,num_of_cores,processor_speed,battery,display,camera,card,os
829,Samsung Galaxy A01 Core,samsung,4999,NaN,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi","(28 nm), Quad Core, 1.5 GHz Processor",False,False,False,"1 GB RAM, 16 GB inbuilt",(28 nm),Quad Core,1.5 GHz Processor,3000 mAh Battery,"5.3 inches, 720 x 1480 px Display",8 MP Rear & 5 MP Front Camera,Memory Card Supported,Android v10


In [60]:
final_df.loc[829,'processor_name'] = 'Mediatek MT6739'


In [61]:
final_df.loc[856]

model                                            Infinix Note 12 VIP
brand_name                                                   infinix
price                                                          19999
rating                                                          84.0
sim                                   Dual Sim, 3G, 4G, VoLTE, Wi-Fi
processor                  Helio  G96, Octa Core, 2.05 GHz Processor
has_5g                                                         False
has_nfc                                                        False
has_blaster                                                    False
ram                                         8 GB RAM, 128 GB inbuilt
processor_name                                            Helio  G96
num_of_cores                                               Octa Core
processor_speed                                   2.05 GHz Processor
battery                     4500 mAh Battery with 120W Fast Charging
display            6.7 inches, 108

In [62]:
final_df.loc[829]

model                            Samsung Galaxy A01 Core
brand_name                                       samsung
price                                               4999
rating                                               NaN
sim                       Dual Sim, 3G, 4G, VoLTE, Wi-Fi
processor          (28 nm), Quad Core, 1.5 GHz Processor
has_5g                                             False
has_nfc                                            False
has_blaster                                        False
ram                              1 GB RAM, 16 GB inbuilt
processor_name                           Mediatek MT6739
num_of_cores                                   Quad Core
processor_speed                        1.5 GHz Processor
battery                                 3000 mAh Battery
display                5.3 inches, 720 x 1480 px Display
camera                     8 MP Rear & 5 MP Front Camera
card                               Memory Card Supported
os                             

In [63]:
processor_brand=final_df['processor_name'].str.split().str.get(0).str.lower()

In [64]:
final_df.insert(11,'processor_brand',processor_brand)

In [65]:
final_df['processor_brand'].value_counts()

processor_brand
snapdragon    408
helio         201
dimensity     177
exynos         49
bionic         39
unisoc         27
tiger          24
google          9
kirin           7
a13             5
qualcomm        5
spreadtrum      4
sc9863a         2
mediatek        1
apple           1
fusion          1
sanpdragon      1
samsung         1
Name: count, dtype: int64

In [66]:
final_df[final_df['processor_brand']=='sanpdragon']

,model,brand_name,price,rating,sim,processor,has_5g,has_nfc,has_blaster,ram,processor_name,processor_brand,num_of_cores,processor_speed,battery,display,camera,card,os
486,Huawei Nova 10 SE,huawei,23999,84.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi, NFC","Sanpdragon 680, Octa Core, 2.4 GHz Processor",False,True,False,"8 GB RAM, 256 GB inbuilt",Sanpdragon 680,sanpdragon,Octa Core,2.4 GHz Processor,4500 mAh Battery with 66W Fast Charging,"6.67 inches, 1080 x 2160 px, 90 Hz Display wit...",108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Supported,Android v12


In [67]:
final_df.loc[486,'processor']='Snapdragon 680, Octa Core, 2.4 GHz Processr'

In [68]:
final_df.loc[486,'processor_brand']='snapdragon'

In [69]:
#final_df[final_df['processor_brand']=='samsung']

In [71]:
final_df['num_of_cores'].value_counts()

num_of_cores
 Octa Core              866
 Hexa Core               31
 Quad Core               31
 Octa Core Processor     19
Octa Core Processor       9
 Hexa Core Processor      8
Quad Core                 6
Octa Core                 5
Name: count, dtype: int64

In [72]:
final_df['num_of_cores']=final_df['num_of_cores'].str.strip()

In [73]:
final_df['num_of_cores']=final_df['num_of_cores'].str.replace('Octa Core Processor','Octa Core').str.replace('Hexa Core Processor','Hexa Core')

In [74]:
# final_df['num_of_cores'].count()

In [75]:
final_df['processor_speed']=final_df['processor_speed'].str.strip().str.split().str.get(0).astype('float')

In [76]:
# name=input('enter your name ')
# print(name[::-1])

# new_name=[]
# for i in name:
#     new_name.append(i)

# new_name.reverse()
# final_name=''.join(new_name)
# print('name after reversing ',final_name)
    

In [77]:
# final_df['ram'].str.split().str.get(0).astype('int')

In [78]:
# final_df['memory']=final_df['ram'].str.split(',').str.get(1).str.split().str.get(0).astype('float')
# final_df['ram_capacity']=final_df['ram'].str.split(',').str.get(0).str.split().str.get(0).astype('int')

In [79]:
ram_capacity=final_df['ram'].str.split(',').str.get(0).str.split().str.get(0).astype('int')
memory=final_df['ram'].str.split(',').str.get(1).str.split().str.get(0).astype('float')

In [80]:
final_df.insert(15,'ram_size',ram_capacity)
final_df.insert(16,'memory',memory)

In [81]:
final_df['ram_size'].value_counts()

ram_size
8      339
6      234
4      215
12      85
3       54
2       32
16       9
1        7
64       3
18       2
512      2
Name: count, dtype: int64

In [82]:
final_df['memory'].value_counts()

memory
128.0    524
64.0     191
256.0    157
32.0      67
512.0     21
16.0      12
1.0        5
4.0        1
8.0        1
Name: count, dtype: int64

In [83]:
temp_df=final_df[final_df['memory']==1]['memory']

final_df.loc[temp_df.index,'memory']=1024

In [84]:
# final_df.loc[temp_df.index]

In [85]:
# final_df.loc[435]['ram_size']

In [86]:
final_df.loc[739,'ram_size']=8

In [87]:
final_df.loc[435,'ram_size']=4

In [88]:
temp_df=final_df.loc[[435,739]]['ram_size']

In [89]:
final_df.loc[temp_df.index,'memory']=temp_df.values

In [90]:
# final_df.drop(614, inplace=True)


In [91]:
# temp_df=final_df[final_df['ram_size']==512]['ram_size']
temp_df=final_df.loc[[478,481],'ram_size']

In [92]:
final_df.loc[temp_df.index,'memory']=temp_df.values


In [93]:
final_df.loc[temp_df.index]

,model,brand_name,price,rating,sim,processor,has_5g,has_nfc,has_blaster,ram,...,processor_brand,num_of_cores,processor_speed,battery,ram_size,memory,display,camera,card,os
478,Huawei Mate 50 RS Porsche Design,huawei,239999,81.0,"Dual Sim, 3G, 4G, VoLTE, Wi-Fi, NFC, IR Blaster","Snapdragon 8+ Gen1, Octa Core, 3.2 GHz Processor",False,True,True,512 GB inbuilt,...,snapdragon,Octa Core,3.2,4700 mAh Battery with 66W Fast Charging,512,512.0,"6.74 inches, 1212 x 2616 px, 120 Hz Display",50 MP + 48 MP + 13 MP Triple Rear & 13 MP Fron...,"Memory Card (Hybrid), upto 256 GB",Hongmeng OS v3.0
481,Nokia 8000 4G,nokia,6899,NaN,"Dual Sim, 3G, 4G, Wi-Fi","Snapdragon 210, Quad Core, 1.1 GHz Processor",False,False,False,"512 MB RAM, 4 GB inbuilt",...,snapdragon,Quad Core,1.1,1500 mAh Battery,512,512.0,"2.8 inches, 240 x 320 px Display",2 MP Rear Camera,Memory Card Supported,NaN


In [94]:
# Set ram_size to NaN at row 478
final_df.loc[478, 'ram_size'] = np.nan

# Set ram_size to 4 at row 481
final_df.loc[481, 'ram_size'] = 4

In [95]:
print(final_df.loc[[478, 481], 'ram_size'])


478    NaN
481    4.0
Name: ram_size, dtype: float64


In [96]:
final_df['battery']=final_df['battery'].str.replace('mAh','')
final_df['battery']=final_df['battery'].str.replace('Battery','')
final_df['battery']=final_df['battery'].str.replace('with','')

In [97]:
battery_capacity=final_df['battery'].str.strip().str.split().str.get(0).astype('float')
charging_speed=final_df['battery'].str.strip().str.split().str.get(1)

In [98]:
final_df.columns

Index(['model', 'brand_name', 'price', 'rating', 'sim', 'processor', 'has_5g',
       'has_nfc', 'has_blaster', 'ram', 'processor_name', 'processor_brand',
       'num_of_cores', 'processor_speed', 'battery', 'ram_size', 'memory',
       'display', 'camera', 'card', 'os'],
      dtype='object')

In [99]:
final_df.insert(18,'battery_capacity',battery_capacity)
final_df.insert(19,'charging_speed',charging_speed)

In [100]:
final_df['charging_speed'].value_counts()

charging_speed
33W      152
18W      128
Fast      68
67W       65
25W       53
120W      46
15W       43
80W       42
66W       37
10W       33
30W       32
65W       30
44W       23
45W       17
20W       10
68W        7
100W       7
150W       7
125W       6
22.5W      5
40W        5
60W        4
55W        3
21W        2
50W        2
210W       2
240W       1
19W        1
200W       1
135W       1
165W       1
180W       1
27W        1
68.2W      1
Name: count, dtype: int64

In [101]:
final_df['battery_capacity'].value_counts()

battery_capacity
5000.0    487
4500.0     97
6000.0     60
4000.0     42
4700.0     27
         ... 
3520.0      1
2150.0      1
5600.0      1
4470.0      1
8000.0      1
Name: count, Length: 91, dtype: int64

In [102]:
temp_df=final_df['charging_speed'][final_df['charging_speed'].str.contains('Fast', na=False)]
# final_df['charging_speed'].value_counts()

In [103]:
final_df.loc[temp_df.index][['model','charging_speed']]

,model,charging_speed
3,Motorola Moto G62 5G,Fast
6,Apple iPhone 14,Fast
15,Apple iPhone 13,Fast
25,Samsung Galaxy S20 FE 5G,Fast
27,Apple iPhone 14 Pro Max,Fast
...,...,...
952,Sharp Aquos R5G,Fast
953,Huawei Mate 30 RS Porsche Design,Fast
957,Samsung Galaxy A9 Pro (2019),Fast
980,Motorola Moto G91 5G,Fast


In [106]:
# len(charging_watts)

In [107]:
charging_watts = [
    15, 20, 20, 25, 20, 20, 20, 67, 15, 10,
    20, 18, 18, 20, 5,  20, 20, 15, 20, 20,
    20, 65, 20, 15, 20, 18, 35, 20, 18, 10,
    18, 18, 25, 18, 44, 20, 25, 20, 18, 45,
    20, 27, 18, 20, 20, 33, 33, 22.5, 15, 15,
    40, 20, 20, 25, 25, 67, 20, 25, 25, 27,
    25, 20, 33, 18, 40, 15, 20, 25
]

final_df.loc[temp_df.index,'charging_speed']=charging_watts


In [108]:
final_df[final_df['charging_speed']=='Fast']

,model,brand_name,price,rating,sim,processor,has_5g,has_nfc,has_blaster,ram,...,processor_speed,battery,ram_size,memory,display,battery_capacity,charging_speed,camera,card,os


In [109]:
temp_df=final_df['charging_speed'][final_df['charging_speed'].isnull()]
# final_df.loc[temp_df.index]

In [110]:
screen_size=final_df['display'].str.strip().str.split(',').str.get(0).str.split().str.get(0).astype('float')

In [111]:
screen_size=final_df['display'].str.strip().str.split(',').str.get(0).str.split().str.get(0)

In [112]:
final_df.insert(21,'screen_size',screen_size)

In [113]:
final_df['screen_size'].count()

982

In [114]:
resolution=final_df['display'].str.strip().str.split(',').str.get(1).str.strip().str.split('px').str.get(0)

In [115]:
final_df.insert(22,'resolution',resolution)

In [116]:
final_df['resolution'].value_counts()

resolution
1080 x 2400     342
720 x 1600      149
1080 x 2408      65
1080 x 2412      58
1080 x 2340      43
               ... 
640 x 960         1
1640 x 720        1
2088 x 2250       1
720 x 1680        1
2220 x 1080       1
Name: count, Length: 84, dtype: int64

In [117]:
refresh_rate = final_df['display'].str.strip().str.split(',').str.get(2).str.strip().str.findall(r'\d{2,3}').str.get(0).apply(lambda x: 60 if pd.isna(x) else x).astype(int)

In [118]:
final_df.insert(23,'refresh_rate',refresh_rate)

In [119]:
final_df['refresh_rate'].value_counts()

refresh_rate
60     370
120    344
90     219
144     39
165      9
240      1
Name: count, dtype: int64

In [120]:
def camera_extractor(text):

  if 'Quad' in text:
    return '4'
  elif 'Triple' in text:
    return '3'
  elif 'Dual' in text:
    return '2'
  elif 'Missing' in text:
    return 'Missing'
  else:
    return '1'

In [121]:
num_rear_cameras = final_df['camera'].str.strip().str.split('&').str.get(0).apply(camera_extractor)

In [122]:
final_df.insert(25,'num_rear_cameras',num_rear_cameras)

In [123]:
final_df['num_rear_cameras'].value_counts()

num_rear_cameras
3    552
2    207
4    156
1     67
Name: count, dtype: int64

In [124]:
num_front_cameras = final_df['camera'].str.strip().str.split('&').str.get(1).str.strip().fillna('Missing').apply(camera_extractor)

In [125]:
final_df.insert(26,'num_front_cameras',num_front_cameras)

In [126]:
final_df['num_front_cameras'].value_counts()

num_front_cameras
1          948
2           29
Missing      5
Name: count, dtype: int64

In [128]:
final_df.columns

Index(['model', 'brand_name', 'price', 'rating', 'sim', 'processor', 'has_5g',
       'has_nfc', 'has_blaster', 'ram', 'processor_name', 'processor_brand',
       'num_of_cores', 'processor_speed', 'battery', 'ram_size', 'memory',
       'display', 'battery_capacity', 'charging_speed', 'camera',
       'screen_size', 'resolution', 'refresh_rate', 'card', 'num_rear_cameras',
       'num_front_cameras', 'os'],
      dtype='object')

In [139]:
final_df=final_df[[
    # 🔹 Basic Info
    'brand_name', 'model', 'price', 'rating', 'os',

    # 🔹 Connectivity
    'sim', 'has_5g', 'has_nfc', 'has_blaster',

    # 🔹 Performance
    'processor_brand', 'processor_name', 'processor', 'num_of_cores', 'processor_speed',
    'ram', 'ram_size', 'memory',

    # 🔹 Display
    'display', 'screen_size', 'resolution', 'refresh_rate',

    # 🔹 Battery
    'battery', 'battery_capacity', 'charging_speed',

    # 🔹 Camera
    'camera', 'num_rear_cameras', 'num_front_cameras',

    # 🔹 Storage Expansion
    'card'
]]


In [141]:
final_df

,brand_name,model,price,rating,os,sim,has_5g,has_nfc,has_blaster,processor_brand,...,screen_size,resolution,refresh_rate,battery,battery_capacity,charging_speed,camera,num_rear_cameras,num_front_cameras,card
0,oneplus,OnePlus 11 5G,54999,89.0,Android v13,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,snapdragon,...,6.7,1440 x 3216,120,5000 100W Fast Charging,5000.0,100W,50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,3,1,Memory Card Not Supported
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,Android v12,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,snapdragon,...,6.59,1080 x 2412,120,5000 33W Fast Charging,5000.0,33W,64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB"
2,samsung,Samsung Galaxy A14 5G,16499,75.0,Android v13,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,exynos,...,6.6,1080 x 2408,90,5000 15W Fast Charging,5000.0,15W,50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,3,1,"Memory Card Supported, upto 1 TB"
3,motorola,Motorola Moto G62 5G,14999,81.0,Android v12,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,snapdragon,...,6.55,1080 x 2400,120,5000 Fast Charging,5000.0,15,50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,"Memory Card (Hybrid), upto 1 TB"
4,realme,Realme 10 Pro Plus,24999,82.0,Android v13,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,dimensity,...,6.7,1080 x 2412,120,5000 67W Fast Charging,5000.0,67W,108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,3,1,Memory Card Not Supported
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,motorola,Motorola Moto Edge S30 Pro,34990,83.0,Android v12,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,snapdragon,...,6.67,1080 x 2460,120,5000 68.2W Fast Charging,5000.0,68.2W,64 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,Memory Card Not Supported
978,honor,Honor X8 5G,14990,75.0,Android v11,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi",True,False,False,snapdragon,...,6.5,720 x 1600,60,5000 22.5W Fast Charging,5000.0,22.5W,48 MP + 2 MP + Depth Sensor Triple Rear & 8 MP...,3,1,"Memory Card Supported, upto 1 TB"
979,poco,POCO X4 GT 5G (8GB RAM + 256GB),28990,85.0,Android v12,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC, IR Bl...",True,True,True,dimensity,...,6.6,1080 x 2460,144,5080 67W Fast Charging,5080.0,67W,64 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,3,1,Memory Card Not Supported
980,motorola,Motorola Moto G91 5G,19990,80.0,Android v12,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC",True,True,False,snapdragon,...,6.8,1080 x 2400,60,5000 Fast Charging,5000.0,20,108 MP + 8 MP + 2 MP Triple Rear & 32 MP Front...,3,1,"Memory Card Supported, upto 1 TB"
